In [ ]:
import random
import nobrainer
import os, sys
sys.path.append('..')
import numpy as np
import nibabel as nb
from glob import glob


import defacing
from defacing import dataloaders, training

In [ ]:
import nobrainer
from nobrainer.io import _is_gzipped
from nobrainer.volume import to_blocks
import sys, os
sys.path.append('../')
from defacing.preprocessing.augmentation import VolumeAugmentations, SliceAugmentations
from defacing.helpers.utils import load_vol
import tensorflow as tf
import glob
import numpy as np

AUTOTUNE = tf.data.experimental.AUTOTUNE

def get_dataset(
    file_pattern,
    n_classes,
    batch_size,
    volume_shape,
    plane,
    n_slices = 24,
    block_shape=None,
    n_epochs=None,
    mapping=None,
    shuffle_buffer_size=None,
    num_parallel_calls=AUTOTUNE,
    mode='train',
):

    """ Returns tf.data.Dataset after preprocessing from
    tfrecords for training and validation

    Parameters
    ----------
    file_pattern:

    n_classes:
    """

    files = glob.glob(file_pattern)

    if not files:
        raise ValueError("no files found for pattern '{}'".format(file_pattern))

    compressed = _is_gzipped(files[0])
    shuffle = bool(shuffle_buffer_size)

    ds = nobrainer.dataset.tfrecord_dataset(
        file_pattern=file_pattern,
        volume_shape=volume_shape,
        shuffle=shuffle,
        scalar_label=True,
        compressed=compressed,
        num_parallel_calls=num_parallel_calls,
    )
    
    
    def _ss(x, y):
        
        x, y = structural_slice(x, y, plane, n_slices)
        return (x, y)
    
    
    ds = ds.map(_ss, num_parallel_calls)
        
    ds = ds.prefetch(buffer_size=batch_size)
    
    if batch_size is not None:
        ds = ds.batch(batch_size=batch_size, drop_remainder=False)
        
    if mode == 'train':
        if shuffle_buffer_size:
            ds = ds.shuffle(buffer_size=shuffle_buffer_size)

#         Repeat the dataset n_epochs times
        ds = ds.repeat(n_epochs)

    return ds


def structural_slice(x, y, plane, n_slices = 4):

    """ Transpose dataset based on the plane

    Parameters
    ----------
    x:

    y:

    plane:
    
    n:

    augment:
    """

    options = ["sagittal", "coronal", "axial", "combined"]
    shape = np.array(x.shape)
    if isinstance(plane, str) and plane in options:
        idxs = np.random.randint(x.shape[0], size=(n_slices, 3))
#         idxs = np.array([[64, 64, 64]])
        if plane == "sagittal":
            midx = idxs[:, 0]
            x = x

        if plane == "coronal":
            midx = idxs[:, 1]
            x = tf.transpose(x, perm=[1, 2, 0])


        if plane == "axial":
            midx = idxs[:, 2]
            x = tf.transpose(x, perm=[2, 0, 1])


        if plane == "combined":
            temp = {}
            for op in options[:-1]:
                temp[op] = structural_slice(x, y, op, n_slices)[0]
            x = temp

        if not plane == "combined":
            x = tf.squeeze(tf.gather_nd(x, midx.reshape(n_slices, 1, 1)), axis=1)
            x = tf.math.reduce_mean(x, axis=0)
            x = tf.expand_dims(x, axis=-1)
            x = tf.convert_to_tensor(x)
        return x, y
    else:
        raise ValueError("expected plane to be one of [sagittal, coronal, axial]")


if __name__ == "__main__":
    ROOTDIR = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_B/128/tfrecords_full'
    n_classes = 2
    global_batch_size = 8
    volume_shape = (128, 128, 128)
    ds = get_dataset(
        os.path.join(ROOTDIR, "data-train_*"),
        n_classes=n_classes,
        batch_size=global_batch_size,
        volume_shape=volume_shape,
        plane="sagittal",
        shuffle_buffer_size=3,
    )
    
    import matplotlib.pyplot as plt

    # x, y = next(ds.as_numpy_operator())
    # print(x.shape, y)
    times = 0
    for x, y in ds.as_numpy_iterator():
        if times == 3:
            break
        print(x.shape, y)
        times += 1

        fig = plt.figure(figsize=(25, 8))

        for i in range(1, 9):
            fig.add_subplot(1,8, i)
            plt.imshow(x[i-1, :, :])


        print(ds)
    
    

In [ ]:
# Std packages
import sys, os
import glob
import math

sys.path.append('..')

# Custom packages
import defacing
from defacing.models import modelN
# from defacing.dataloaders.dataset import get_dataset

# Tf packages
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    LearningRateScheduler,
    TensorBoard,
    EarlyStopping,
)
from tensorflow.keras import metrics
from tensorflow.keras import losses


def scheduler(epoch):
    if epoch < 3:
        return 0.001
    else:
        return 0.001 * tf.math.exp(0.1 * (10 - epoch))


def train(
    csv_path,
    model_save_path,
    tfrecords_path,
    volume_shape=(64, 64, 64),
    image_size=(64, 64),
    dropout=0.2,
    batch_size=16,
    n_slices=16,
    n_classes=2,
    n_epochs=15,
    percent=100,
    mode='CV',
):
    
    
    train_csv_path = os.path.join(csv_path, "training.csv")
    train_paths = pd.read_csv(train_csv_path)["X"].values
    train_labels = pd.read_csv(train_csv_path)["Y"].values
    
    if mode == 'CV':
        valid_csv_path = os.path.join(csv_path, "validation.csv")
        valid_paths = pd.read_csv(valid_csv_path)["X"].values
        valid_labels = pd.read_csv(valid_csv_path)["Y"].values
    
    weights = class_weight.compute_class_weight('balanced',
                                                np.unique(train_labels),
                                                train_labels)
    weights = dict(enumerate(weights))
    
    print(weights)
    
    planes = ["axial", "coronal", "sagittal", "combined"]
    

    global_batch_size = batch_size
    
    os.makedirs(model_save_path, exist_ok=True)
    cp_save_path = os.path.join(model_save_path, "weights")
    logdir_path = os.path.join(model_save_path, "tb_logs")
    metrics_path = os.path.join(model_save_path, "metrics")
    
    os.makedirs(metrics_path, exist_ok=True)
#     os.makedirs(logdir_path, exist_ok=True)
        
    for plane in planes:

        logdir = os.path.join(logdir_path, plane)
        os.makedirs(logdir, exist_ok=True)

        tbCallback = TensorBoard(log_dir=logdir)

        os.makedirs(os.path.join(cp_save_path, plane), exist_ok=True)

        model_checkpoint = ModelCheckpoint(
            os.path.join(cp_save_path, plane, "best-wts.h5"),
            monitor="val_loss",
            save_weights_only=True,
            mode="min",
        )

#         with strategy.scope():

        if not plane == "combined": 
            lr = 1e-3
            model = modelN.Submodel(
                input_shape=image_size,
                dropout=dropout,
                name=plane,
                include_top=True,
                weights=None,
            )
        else:
            lr = 5e-4
            model = modelN.CombinedClassifier(
                input_shape=image_size,
                dropout=dropout,
                trainable=True,
                wts_root=cp_save_path,
            )

        print("Submodel: ", plane)
#         print(model.summary())

        METRICS = [
            metrics.TruePositives(name="tp"),
            metrics.FalsePositives(name="fp"),
            metrics.TrueNegatives(name="tn"),
            metrics.FalseNegatives(name="fn"),
            metrics.BinaryAccuracy(name="accuracy"),
            metrics.Precision(name="precision"),
            metrics.Recall(name="recall"),
            metrics.AUC(name="auc"),
        ]

        model.compile(
            loss=tf.keras.losses.binary_crossentropy,
            optimizer=Adam(learning_rate=lr),
            metrics=METRICS,
        )

        print("GLOBAL BATCH SIZE: ", global_batch_size)

        dataset_train = get_dataset(
            file_pattern=os.path.join(tfrecords_path, 'data-train_*'),
            n_classes=n_classes,
            batch_size=global_batch_size,
            volume_shape=volume_shape,
            plane=plane,
            n_slices=n_slices,
            shuffle_buffer_size=global_batch_size,
        )
        
        steps_per_epoch = math.ceil(len(train_paths)/batch_size)
        print(steps_per_epoch)
        
        # CALLBACKS
        lrcallback = tf.keras.callbacks.LearningRateScheduler(scheduler)
        
        if mode == 'CV':
            earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
            
            dataset_valid = get_dataset(
                file_pattern=os.path.join(tfrecords_path, "data-valid_*"),
                n_classes=n_classes,
                batch_size=global_batch_size,
                volume_shape=volume_shape,
                plane=plane,
                n_slices=n_slices,
                shuffle_buffer_size=global_batch_size,
            )
            
            validation_steps = math.ceil(len(valid_paths)/batch_size)
            
            history = model.fit(
                dataset_train,
                epochs=n_epochs,
                steps_per_epoch=steps_per_epoch,
                validation_data=dataset_valid,
                validation_steps=validation_steps,
                callbacks=[tbCallback, model_checkpoint],
                class_weight = weights,
            )
            
            hist_df =  pd.DataFrame(history.history)
            
        else:
            earlystopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
            print(model.summary())
            print("Steps/Epoch: ", steps_per_epoch)
            history = model.fit(
                dataset_train,
                epochs=n_epochs,
                steps_per_epoch=steps_per_epoch,
                callbacks=[tbCallback, model_checkpoint, earlystopping],
                class_weight = weights,
            )
            
        hist_df =  pd.DataFrame(history.history)
        jsonfile = os.path.join(metrics_path, plane + '.json')
        
        with open(jsonfile, mode='w') as f:
            hist_df.to_json(f)
        
        del model
        K.clear_session()
        
    return history


# if __name__ == "__main__":
#     ROOTDIR = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_B/128'
#     csv_path = os.path.join(ROOTDIR, "csv_full")
#     model_save_path = os.path.join(ROOTDIR, "model_save_dir_full")
#     tfrecords_path = os.path.join(ROOTDIR, 'tfrecords_full')
    
#     history = train(
#     csv_path,
#     model_save_path,
#     tfrecords_path,
#     volume_shape=(128, 128, 128),
#     image_size=(128, 128),
#     mode='full'
#     )
    


In [ ]:
# Std packages
import sys, os
import glob
import math
import tensorflow as tf

sys.path.append('..')

# Custom packages
# import defacing
# from defacing.models import modelN
# from defacing.dataloaders.dataset import get_dataset
# from defacing.training.training import train

for fold in range(8, 16):
    ROOTDIR = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_B/128'
    
    csv_path = os.path.join(ROOTDIR, 'csv_F15/train_test_fold_{}/csv'.format(fold))
    model_save_path = os.path.join(ROOTDIR, 'model_save_dir_F15/train_test_fold_{}'.format(fold))
    tfrecords_path = os.path.join(ROOTDIR, 'tfrecords_F15/tfrecords_fold_{}'.format(fold))
    
    history = train(
        csv_path,
        model_save_path,
        tfrecords_path,
        volume_shape = (128, 128, 128),
        image_size = (128, 128),
        mode = 'CV',
        n_slices=24,
        batch_size = 32,
    )

In [ ]:
pip install -U tensorboard_plugin_profile


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


In [ ]:
from nondefaced_detector import preprocess
vol_path = '../../examples/sample_vols/IXI002-Guys-0828-T1.nii.gz'
save_path = ''
ppath, cpath = preprocess.preprocess(vol_path, save_path=save_path)
print(ppath, cpath)

In [ ]:
import sys, os
from nondefaced_detector.models.modelN import CombinedClassifier
from nondefaced_detector.helpers import utils

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics
from tensorflow.keras import losses

weights_path = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_B/128/model_save_dir_full/weights/combined/best-wts.h5'
wts_root = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_B/128/model_save_dir_full/weights'

model = CombinedClassifier(
        input_shape=(128, 128), dropout=0.4, wts_root=wts_root, trainable=False)
    
model.load_weights(os.path.abspath(weights_path))
model.trainable = False

METRICS = [
        metrics.TruePositives(name="tp"),
        metrics.FalsePositives(name="fp"),
        metrics.TrueNegatives(name="tn"),
        metrics.FalseNegatives(name="fn"),
        metrics.BinaryAccuracy(name="accuracy"),
        metrics.Precision(name="precision"),
        metrics.Recall(name="recall"),
        metrics.AUC(name="auc"),
]

# model.compile(
#         loss=tf.keras.losses.binary_crossentropy,
#         optimizer=Adam(learning_rate=1e-3),
#         metrics=METRICS,
#     )

volume, affine, _ = utils.load_vol(cpath)

print(volume.shape)

# dataset_test = get_dataset(
#         file_pattern=,
#         n_classes=2,
#         batch_size=128,
#         volume_shape=(128, 128, 128),
#         plane='combined',
#         mode='test'
#     )


# print(volume.shape)

# model.predict(volume)

    


In [29]:
"""Methods to predict using trained models"""

import os
import numpy as np
import tensorflow as tf

from pathlib import Path

from nondefaced_detector.models.modelN import CombinedClassifier
from nondefaced_detector.helpers       import utils


def predict(
    input_volume,
    model_path,
    batch_size=1,
    n_samples=1,
    n_slices=32,
):
    """Return predictions from `inputs`.

    This is a general prediction method.

    Parameters
    ---------


    Returns
    ------
    """

    if n_samples < 1:
        raise Exception("n_samples cannot be lower than 1.")

    model = _get_model(model_path)

    ds = _structural_slice(input_volume, plane='combined', n_slices=n_slices)
    ds = tf.data.Dataset.from_tensor_slices(ds)
    ds = ds.batch(batch_size=batch_size, drop_remainder=False)

    predicted = model.predict(ds)

    return predicted


def _structural_slice(x, plane, n_slices=16):

    """Transpose dataset based on the plane

    Parameters
    ----------
    x:

    plane:

    n_slices:

    Returns
    -------
    """

    options = ["sagittal", "coronal", "axial", "combined"]

    if isinstance(plane, str) and plane in options:
        idxs = np.random.randint(x.shape[0], size=(n_slices, 3))
        if plane == "sagittal":
            midx = idxs[:, 0]
            x = x

        if plane == "coronal":
            midx = idxs[:, 1]
            x = tf.transpose(x, perm=[1, 2, 0])

        if plane == "axial":
            midx = idxs[:, 2]
            x = tf.transpose(x, perm=[2, 0, 1])

        if plane == "combined":
            temp = {}
            for op in options[:-1]:
                temp[op] = _structural_slice(x, op, n_slices)
            x = temp

        if not plane == "combined":
            x = tf.squeeze(tf.gather_nd(x, midx.reshape(n_slices, 1, 1)), axis=1)
            x = tf.math.reduce_mean(x, axis=0, keepdims=True)
            x = tf.expand_dims(x, axis=-1)
            x = tf.convert_to_tensor(x)

        return x
    else:
        raise ValueError("Expected plane to be one of [sagittal, coronal, axial, combined]")


def _get_model(model_path):

    """Return `tf.keras.Model` object from a filepath.

    Parameters
    ----------
    path: str, path to HDF5 or SavedModel file.

    Returns
    -------
    Instance of `tf.keras.Model`.

    Raises
    ------
    `ValueError` if cannot load model.
    """

    try:
        model = CombinedClassifier(
            input_shape=(128, 128), wts_root=model_path, trainable=False
        )
        
        p = Path(model_path).resolve()
        
        combined_weights = list(
            Path(os.path.join(p, 'combined')).glob('*.h5')
        )[0].resolve()
        

        model.load_weights(combined_weights)
        model.trainable = False

        return model

    except Exception as e:
        print(e)
        pass

    raise ValueError("Failed to load model.")


if __name__=="__main__":

    from nondefaced_detector import preprocess
    from nondefaced_detector.helpers import utils

#     weights_path = 'models/pretrained_weights/combined/best-wts.h5'
    wts_root = '../../nondefaced_detector/models/pretrained_weights'
    model = _get_model(wts_root)
    
#     vol_path = '../examples/sample_vols/IXI002-Guys-0828-T1.nii.gz'
#     ppath, cpath = preprocess.preprocess(vol_path)

#     volume, affine,_ = utils.load_vol(cpath)

#     predicted = predict(volume, wts_root, weights_path)

#     print(predicted)

Model: "model_146"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
axial (InputLayer)              [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
sagittal (InputLayer)           [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
coronal (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_732 (Conv2D)             (None, 128, 128, 8)  80          axial[0][0]                      
__________________________________________________________________________________________

[[0.9999958]]


In [6]:
!which python3

/usr/bin/python3


In [ ]:
import sys, os
sys.path.append('..')
from defacing.models.modelN import CombinedClassifier
# from defacing.dataloaders.dataset import get_dataset


# Tf packages
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics
from tensorflow.keras import losses

def inference(tfrecords_path, weights_path, wts_root):
    
    model = CombinedClassifier(
        input_shape=(128, 128), dropout=0.4, wts_root=wts_root, trainable=False)
    
    model.load_weights(os.path.abspath(weights_path))
    model.trainable = False
    
    dataset_test = get_dataset(
        file_pattern=os.path.join(tfrecords_path, "data-test_*"),
        n_classes=2,
        batch_size=128,
        volume_shape=(128, 128, 128),
        plane='combined',
        mode='test'
    )

    METRICS = [
        metrics.TruePositives(name="tp"),
        metrics.FalsePositives(name="fp"),
        metrics.TrueNegatives(name="tn"),
        metrics.FalseNegatives(name="fn"),
        metrics.BinaryAccuracy(name="accuracy"),
        metrics.Precision(name="precision"),
        metrics.Recall(name="recall"),
        metrics.AUC(name="auc"),
    ]
    
    model.compile(
        loss=tf.keras.losses.binary_crossentropy,
        optimizer=Adam(learning_rate=1e-3),
        metrics=METRICS,
    )
    eval_dict = model.evaluate(dataset_test, return_dict=True)
    predictions = (model.predict(dataset_test) > 0.5).astype(int)
    
    
    return eval_dict, predictions


ROOTDIR = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/test_ixi'
tfrecords_path = os.path.join(ROOTDIR, "tfrecords_new")
weights_path = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_B/128/model_save_dir_full/weights/combined/best-wts.h5'
wts_root = '/tf/shank/HDDLinux/Stanford/data/mriqc-shared/experiments/experiment_B/128/model_save_dir_full/weights'
edict, preds = inference(tfrecords_path, weights_path, wts_root)


In [ ]:
len(preds)
print(edict)

In [ ]:
import matplotlib.pyplot as plt

# x, y = next(ds.as_numpy_operator())
# print(x.shape, y)
times = 0
for x, y in ds.as_numpy_iterator():
    if times == 3:
        break
    print(x.shape, y)
    times += 1
    
    fig = plt.figure(figsize=(25, 8))
    
    for i in range(1, 9):
        fig.add_subplot(1,8, i)
        plt.imshow(x[i-1, :, :])
    
#     all_imgs = []
#     for i in range(len(batch_predictions)):
#         if batch_predictions.flatten()[i] != y.flatten()[i].astype(int):
#             incorr += 1
#             print("Predicted: ",batch_predictions.flatten()[i], "Actual: ", y.flatten()[i].astype(int))
#         else:
#             corr += 1
            
#             fig = plt.figure(figsize=(25, 8))
#             rows, cols = 3, 16
            
#             for i in range(1, cols*rows + 1):
# #                 if i/cols == 1:
# #                     use = x['coronal']
# #                 if i/cols == 2:
# #                     use = x['sagittal']
                    
#                 fig.add_subplot(rows, cols, i)
                
#                 plt.imshow(use[(i-1)%cols,:,:, 0])


#             plt.show()